In [80]:
import json
import argparse
import glob
import sys
sys.path.append("../")
from tool import finqa_equal
from typing import Dict, Any


In [81]:
name = 'convfinqa_direct_gpt-4_04_12_20_53.jsonl'

In [82]:
exception = []
correct, wrong, missing = 0, 0, 0
with open(name) as f:
    for i, line in enumerate(f):
        data = json.loads(line)
        if 'demonstration' in data or 'prompt' in data:
            continue
        else:
            # Select the prediction field.
            if 'executed' in data:
                prediction = data['executed']
            else:
                prediction = data['prediction']
                if False:
                    # For multi-choise question, pick A option
                    if prediction == 'None':
                        prediction = 'A'
            # Calculate the evaluation score
            pred = finqa_equal(prediction, data['answer'], True, True)
            if prediction is None:
                missing += 1
            elif pred:
                correct += 1
            else:
                exception.append(data)
                wrong += 1

print('file name: ', name)
print('num of examples: ', correct + wrong + missing)
print('num of missing returns', missing)
print('accuracy: ', correct / (correct + wrong + missing))

file name:  convfinqa_direct_gpt-4_04_12_20_53.jsonl
num of examples:  421
num of missing returns 0
accuracy:  0.6247030878859857


In [83]:
import pandas as pd

In [84]:
incorrect = pd.DataFrame(exception)

In [85]:
#prompt_msg_4shot = [{"role":"system","content":"You are an AI assistant that helps people find information."},{"role":"user","content":"Read the following text and table, and then answer the last question in a series of questions:\n- | shares available for awards | shares subject to outstanding awards\n2009 global incentive plan | 2322450 | 2530454\n2004 stock incentive plan | - | 5923147\nQuestions: how many shares are subject to outstanding awards is under the 2009 global incentive plan? what about under the 2004 stock incentive plan? how many total shares are subject to outstanding awards? what about under the 2004 stock incentive plan?\nQuestion: what proportion does this represent?"},{"role":"assistant","content":"#Python\nshares_subject_to_outstanding_awards_2009_global_incentive_plan = 2530454\nshares_subject_to_outstanding_awards_2004_stock_incentive_plan = 5923147\ntotal_shares_subject_to_outstanding_awards = shares_subject_to_outstanding_awards_2009_global_incentive_plan + shares_subject_to_outstanding_awards_2004_stock_incentive_plan\nproportion = shares_subject_to_outstanding_awards_2009_global_incentive_plan / total_shares_subject_to_outstanding_awards\nans = proportion"},{"role":"user","content":"Read the following text and table, and then answer the last question in a series of questions:\ncompensation expense the company recorded $ 43 million , $ 34 million , and $ 44 million of expense related to stock awards for the years ended december 31 , 2015 , 2014 , and 2013 , respectively . \nQuestions: what is the compensation expense the company recorded in 2015? what about in 2014? what is the total compensation expense the company recorded in 2015 and 2014? what is the total expenses including 2013?\nQuestion: what is the average for three years?"},{"role":"assistant","content":"#Python\ncompensation_expense_2015 = 43\ncompensation_expense_2014 = 34\ncompensation_expense_2013 = 44\ntotal_compensation_expense = compensation_expense_2015 + compensation_expense_2014 + compensation_expense_2013\naverage_for_three_years = total_compensation_expense / 3\nans = average_for_three_years\n"},{"role":"user","content":"Read the following text and table, and then answer the last question in a series of questions:\nthe net loss on disposal of those assets was $ 344000 for 2005 and $ 43000 for 2004 . \nQuestions: what was the net loss on disposal of assets in 2005? what was the value in 2004? what was the change in value?\nQuestion: what was the percent change?"},{"role":"assistant","content":"#Python\nnet_loss_on_disposal_of_assets_2005 = 344000\nnet_loss_on_disposal_of_assets_2004 = 43000\nnet_change_in_value = net_loss_on_disposal_of_assets_2005 - net_loss_on_disposal_of_assets_2004\npercent_change = net_change_in_value / net_loss_on_disposal_of_assets_2004\nans = percent_change\n"},{"role":"user","content":"Read the following text and table, and then answer the last question in a series of questions:\nlocation | operations conducted | approximatesquare feet | leaseexpirationdates\ndublin ireland | global supply chain distribution and administration offices | 160000 | owned\nathlone ireland | commercial research and development manufacturing | 80000 | owned\nbogart georgia | commercial research and development manufacturing | 70000 | owned\nsmithfield rhode island | commercial research and development manufacturing | 67000 | owned\nQuestions: what is the square feet of the owned global supply chain distribution and administration offices? what is the square feet of the owned commercial research and development manufacturing? what is the sum of those values? what is the total sum including square feet of commercial research and development manufacturing in bogart, georgia? what is the total sum including square feet of commercial research and development manufacturing in smithfield, rhode island?\nQuestion: what is the total sum of square feet owned?"},{"role":"assistant","content":"#Python\nsquare_feet_owned = [160000, 80000, 70000, 67000]\ntotal_square_feet_owned = sum(square_feet_owned)\nans = total_square_feet_owned"},{"role":"user","content":"Read the following text and table, and then answer the last question in a series of questions:  \nentergy new orleans estimates that approximately 141000 electric customers and 93000 gas customers have returned since hurricane katrina and are taking service as of december 31 , 2008 , compared to approximately 132000 electric customers and 86000 gas customers as of december 31 , 2007 .  \nQuestions: what was the number of gas customers in 2008? and what was it in 2007? what was, then, the change in that number over the year?  \nQuestion: and how much does this change represent in relation to the 2007 number of customers, in percentage?"},{"role":"assistant","content":"#Python\ngas_customers_2008 = 93000\ngas_customers_2007 = 86000\nchange_in_gas_customers = gas_customers_2008 - gas_customers_2007\npercent_change = (change_in_gas_customers / gas_customers_2007) / gas_customers_2007\nans = percent_change"}]
prompt_msg_4shot = [{"role":"system","content":"You are an AI assistant that helps people find information. You will be given a context a question and will be asked to provide an answer in the form of a program.  Ensure the program has a variable called 'ans' which provides the answer to the last question."},{"role":"user","content":"Read the following text and table, and then answer the last question in a series of questions:\n- | shares available for awards | shares subject to outstanding awards\n2009 global incentive plan | 2322450 | 2530454\n2004 stock incentive plan | - | 5923147\nQuestions: how many shares are subject to outstanding awards is under the 2009 global incentive plan? what about under the 2004 stock incentive plan? how many total shares are subject to outstanding awards? what about under the 2004 stock incentive plan?\nQuestion: what proportion does this represent?"},{"role":"assistant","content":"#Python\nshares_subject_to_outstanding_awards_2009_global_incentive_plan = 2530454\nshares_subject_to_outstanding_awards_2004_stock_incentive_plan = 5923147\ntotal_shares_subject_to_outstanding_awards = shares_subject_to_outstanding_awards_2009_global_incentive_plan + shares_subject_to_outstanding_awards_2004_stock_incentive_plan\nproportion = shares_subject_to_outstanding_awards_2009_global_incentive_plan / total_shares_subject_to_outstanding_awards\nans = proportion"},{"role":"user","content":"Read the following text and table, and then answer the last question in a series of questions:\ncompensation expense the company recorded $ 43 million , $ 34 million , and $ 44 million of expense related to stock awards for the years ended december 31 , 2015 , 2014 , and 2013 , respectively . \nQuestions: what is the compensation expense the company recorded in 2015? what about in 2014? what is the total compensation expense the company recorded in 2015 and 2014? what is the total expenses including 2013?\nQuestion: what is the average for three years?"},{"role":"assistant","content":"#Python\ncompensation_expense_2015 = 43\ncompensation_expense_2014 = 34\ncompensation_expense_2013 = 44\ntotal_compensation_expense = compensation_expense_2015 + compensation_expense_2014 + compensation_expense_2013\naverage_for_three_years = total_compensation_expense / 3\nans = average_for_three_years\n"},{"role":"user","content":"Read the following text and table, and then answer the last question in a series of questions:\nthe net loss on disposal of those assets was $ 344000 for 2005 and $ 43000 for 2004 . \nQuestions: what was the net loss on disposal of assets in 2005? what was the value in 2004? what was the change in value?\nQuestion: what was the percent change?"},{"role":"assistant","content":"#Python\nnet_loss_on_disposal_of_assets_2005 = 344000\nnet_loss_on_disposal_of_assets_2004 = 43000\nnet_change_in_value = net_loss_on_disposal_of_assets_2005 - net_loss_on_disposal_of_assets_2004\npercent_change = net_change_in_value / net_loss_on_disposal_of_assets_2004\nans = percent_change\n"},{"role":"user","content":"Read the following text and table, and then answer the last question in a series of questions:\nlocation | operations conducted | approximatesquare feet | leaseexpirationdates\ndublin ireland | global supply chain distribution and administration offices | 160000 | owned\nathlone ireland | commercial research and development manufacturing | 80000 | owned\nbogart georgia | commercial research and development manufacturing | 70000 | owned\nsmithfield rhode island | commercial research and development manufacturing | 67000 | owned\nQuestions: what is the square feet of the owned global supply chain distribution and administration offices? what is the square feet of the owned commercial research and development manufacturing? what is the sum of those values? what is the total sum including square feet of commercial research and development manufacturing in bogart, georgia? what is the total sum including square feet of commercial research and development manufacturing in smithfield, rhode island?\nQuestion: what is the total sum of square feet owned?"},{"role":"assistant","content":"#Python\nsquare_feet_owned = [160000, 80000, 70000, 67000]\ntotal_square_feet_owned = sum(square_feet_owned)\nans = total_square_feet_owned"},{"role":"user","content":"Read the following text and table, and then answer the last question in a series of questions:  \nentergy new orleans estimates that approximately 141000 electric customers and 93000 gas customers have returned since hurricane katrina and are taking service as of december 31 , 2008 , compared to approximately 132000 electric customers and 86000 gas customers as of december 31 , 2007 .  \nQuestions: what was the number of gas customers in 2008? and what was it in 2007? what was, then, the change in that number over the year?  \nQuestion: and how much does this change represent in relation to the 2007 number of customers, in percentage?"},{"role":"assistant","content":"#Python\ngas_customers_2008 = 93000\ngas_customers_2007 = 86000\nchange_in_gas_customers = gas_customers_2008 - gas_customers_2007\npercent_change = (change_in_gas_customers / gas_customers_2007) / gas_customers_2007\nans = percent_change"}]

In [86]:
def create_reader_request_processed(example: Dict[str, Any]):
    prompt = 'Read the following text and table, and then answer the last question in a series of questions:\n'
    if example['golden_text']:
        prompt += example['golden_text'].strip() + '\n'
    if example['golden_table']:
        prompt += example['golden_table'].strip() + '\n'
    #prompt += '\n'
    prompt += 'Questions: '
    prompt += " ".join(example['questions'][:-1])
    prompt += '\n'
    prompt += f'Question: {example["questions"][-1]}'
    prompt += '\n'
    prompt += 'Answer:'
    return prompt

In [87]:
def create_reader_request_processed_chat(example: Dict[str, Any]):
    prompt = 'Read the following text and table, and then answer the last question in a series of questions:\n'
    if example['golden_text']:
        prompt += example['golden_text'].strip() + '\n'
    if example['golden_table']:
        prompt += example['golden_table'].strip() + '\n'
    #prompt += '\n'
    prompt += 'Questions: '
    prompt += " ".join(example['questions'][:-1])
    prompt += '\n'
    prompt += f'Question: {example["questions"][-1]}'
    return prompt

In [88]:
incorrect['chatprompt'] = incorrect.apply(lambda x:  prompt_msg_4shot + [{"role":"user","content":create_reader_request_processed_chat(x)}], axis=1)

In [89]:
incorrect.columns

Index(['questions', 'answer', 'text', 'table', 'golden_text', 'golden_table',
       'id', 'programs', 'generated', 'executed', 'chatprompt'],
      dtype='object')

In [90]:
incorrect

,questions,answer,text,table,golden_text,golden_table,id,programs,generated,executed,chatprompt
0,[what was the change in the unamortized debt i...,0.375,"as of december 31 , 2017 , the company had gro...",( $ in millions ) | december 31 2017 | decembe...,the company had unamortized debt issuance cost...,( $ in millions ) | december 31 2017 | decembe...,Double_HII/2017/page_104.pdf,"[ans = 15 - 19, x0 = 15 - 19; ans = x0 / 19, a...",[#Python\nunamortized_debt_issuance_costs_seni...,142.402546,"[{'role': 'system', 'content': 'You are an AI ..."
1,[how much did the gallons hedged in 2018 repre...,14.0,"republic services , inc . notes to consolidate...",year | gallons hedged | weighted average contr...,the aggregate fair values of our outstanding f...,year | gallons hedged | weighted average contr...,Double_RSG/2016/page_144.pdf,"[ans = 12000000 / 3000000, ans = 37.8, ans = 2...",[#Python\ngallons_hedged_2017 = 12000000\ngall...,0.071429,"[{'role': 'system', 'content': 'You are an AI ..."
2,[what is the change in price of the s&p 500 fr...,18770.0,"market for registrant 2019s common equity , re...",- | 12/28/2013 | 1/3/2015 | 1/2/2016 | 12/31/2...,,- | 12/28/2013 | 1/3/2015 | 1/2/2016 | 12/31/2...,Single_CDNS/2018/page_31.pdf-1,"[ans = 129.05 - 110.28, x0 = 129.05 - 110.28; ...",[#Python\nprice_2015 = 110.28\nprice_2016 = 10...,-740.0,"[{'role': 'system', 'content': 'You are an AI ..."
3,[what were the total accumulated other compreh...,0.62912,accumulated other comprehensive losses : pmi's...,( losses ) earnings ( in millions ) | ( losses...,,( losses ) earnings ( in millions ) | ( losses...,Double_PM/2015/page_127.pdf,"[ans = 9402, ans = 6826, ans = 9402 - 6826, x0...",[#Python\ntotal_accumulated_losses_2015 = 9402...,0.014081,"[{'role': 'system', 'content': 'You are an AI ..."
4,[what was the total expense for repairs and ma...,2525.0,are allocated using appropriate statistical ba...,millions | dec . 31 2011 | dec . 31 2010\nacco...,total expense for repairs and maintenance incu...,millions | dec . 31 2011 | dec . 31 2010\nacco...,Single_UNP/2011/page_76.pdf-1,"[ans = 2.2 * 1000, ans = 325, x0 = 2.2 * 1000;...",[#Python\ntotal_expense_for_repairs_and_mainte...,2525000000.0,"[{'role': 'system', 'content': 'You are an AI ..."
...,...,...,...,...,...,...,...,...,...,...,...
153,[what was the amount of long-term debt maturit...,0.07039,entergy corporation and subsidiaries notes to ...,- | amount ( in thousands )\n2016 | $ 204079\n...,entergy arkansas is the only entergy company t...,- | amount ( in thousands )\n2018 | $ 822690\n...,Single_ETR/2015/page_131.pdf-3,"[ans = 822690, ans = 768588, ans = 822690 - 76...",[#Python\nlong_term_debt_maturities_2018 = 822...,6.576232,"[{'role': 'system', 'content': 'You are an AI ..."
154,"[what was the value of liabilities assumed?, w...",0.17792,use of estimates the preparation of the financ...,operating rental properties | $ 602011\nland h...,,operating rental properties | $ 602011\nother ...,Single_DRE/2007/page_59.pdf-1,"[ans = 5829, ans = 148527, ans = 5829 + 148527...",[#Python\nliabilities_assumed = -5829\npositiv...,0.164482,"[{'role': 'system', 'content': 'You are an AI ..."
155,"[between the years of 2017 and 2018, what was ...",0.0,table of contents adobe inc . notes to consoli...,- | 2018 | 2017\nbeginning balance | $ 172945 ...,to the extent that the foreign earnings previo...,- | 2018 | 2017\nending balance | $ 196152 | $...,Double_ADBE/2018/page_86.pdf,"[ans = 196152 - 172945, ans = 172945, x0 = 196...",[#Python\nunrecognized_tax_benefits_2018 = 196...,0.074906,"[{'role': 'system', 'content': 'You are an AI ..."
156,[how much does the total of intangibles repres...,80.0,devon energy corporation and subsidiaries note...,year | amortization amount ( in millions )\n20...,amortization expense for intangibles was appro...,year | amortization amount ( in millions )\n20...,Single_DVN/2014/page_85.pdf-2,"[ans = 36 / 45, x0 = 36 / 45; ans = x0 * 100]",[#Python\namortization_expense_2014

In [94]:
i=0
#print(incorrect['text'][i])
print(incorrect['chatprompt'][i])
print('--------')
print(incorrect['questions'][i])
print('--------')
print(incorrect['generated'][i][0])
print('--------')
print(incorrect['golden_text'][i])
print('--------')
print(incorrect['golden_table'][i])
print('--------')
print(incorrect['programs'][i])
print(incorrect['answer'][i])


[{'role': 'system', 'content': "You are an AI assistant that helps people find information. You will be given a context a question and will be asked to provide an answer in the form of a program.  Ensure the program has a variable called 'ans' which provides the answer to the last question."}, {'role': 'user', 'content': 'Read the following text and table, and then answer the last question in a series of questions:\n- | shares available for awards | shares subject to outstanding awards\n2009 global incentive plan | 2322450 | 2530454\n2004 stock incentive plan | - | 5923147\nQuestions: how many shares are subject to outstanding awards is under the 2009 global incentive plan? what about under the 2004 stock incentive plan? how many total shares are subject to outstanding awards? what about under the 2004 stock incentive plan?\nQuestion: what proportion does this represent?'}, {'role': 'assistant', 'content': '#Python\nshares_subject_to_outstanding_awards_2009_global_incentive_plan = 2530